In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.interpolate import PchipInterpolator
from scipy.integrate import quad
import plotly.graph_objects as go
import numpy as np

# Level-1 Trigger (CMS)

In [2]:
egamma = pd.read_csv("l1t_data/egamma.csv")
tau = pd.read_csv("l1t_data/isolated tau.csv")
jet_energy = pd.read_csv("l1t_data/jet energy.csv")
muon = pd.read_csv("l1t_data/single muon.csv")

In [3]:
rates = pd.read_excel("l1t_data/trigger_rates.xlsx")

In [4]:
triggers = ["Jet", "Muon", "EGamma", "Tau"]
jet_rate = rates["Jet Rate"][0]
muon_rate = rates["Muon Rate"][0]
egamma_rate = rates["Egamma Rate"][0]
tau_rate = rates["Tau Rate"][0]
all_rates = [jet_rate, muon_rate, egamma_rate, tau_rate]

In [5]:
fig = go.Figure()
fig.add_bar(x = triggers, y = all_rates)
fig.update_layout(width = 800, height = 600)

In [6]:
exp_dist = lambda x, l: l * np.exp(-1 * l * x) * (x > 0)

In [7]:
exp_cdf = lambda x, l: 1 - np.exp(-l * x)

In [8]:
exp_generator = lambda p, l: (-1 / l) * np.log(1 - p)

In [9]:
xs = np.linspace(0.01, 5.0, 101)
fig = go.Figure()
fig.add_scatter(x = xs, y = exp_dist(xs, 1.5), name = "PDF")
fig.add_scatter(x = xs, y = exp_cdf(xs, 1.5), name = "CDF")
fig.update_layout(width = 800, height = 600)

In [10]:
data_range = lambda x: np.linspace(np.min(x), np.max(x), 101)

In [24]:
egamma_spl = PchipInterpolator(egamma["x"], egamma[" y"])

In [25]:
egamma_xs = data_range(egamma["x"])

In [26]:
egamma_rate

0.444

# Fit to EGamma L1T

In [32]:
#fit to muon - minimize the difference between the trigger efficiency multiplied by the underlying distribution and the trigger rate
egamma_fit = lambda l: np.abs(egamma_rate - quad(lambda x: exp_dist(x, l) * egamma_spl(x), np.min(egamma_xs), np.max(egamma_xs))[0])

In [33]:
egamma_soln = minimize_scalar(egamma_fit, bounds = [0.00, 0.40], method="bounded")
egamma_l = egamma_soln.x

/var/folders/qc/b4ggx3516_qc8fy84rgtp36c0000gq/T/ipykernel_69534/3527769200.py:2: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



In [34]:
egamma_soln

     fun: 1.6358825408624433e-05
 message: 'Solution found.'
    nfev: 18
     nit: 18
  status: 0
 success: True
       x: 0.02585292794736181

In [35]:
egamma_ideal = lambda x: 1.0 * (x > 30)

In [38]:
fig = go.Figure()
fig.add_scatter(x = egamma["x"], y = egamma[" y"], name="Data")
fig.add_scatter(x = egamma_xs, y = egamma_spl(egamma_xs), name="Interpolation")
fig.add_scatter(x = egamma_xs, y = egamma_ideal(egamma_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = egamma_xs, y = 1.0 - exp_cdf(egamma_xs, egamma_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "e-gamma L1T efficiency",
                  xaxis_title = "Momentum (GeV)",
                  yaxis_title = "Efficiency",
                  )
fig.update_xaxes(type = "log")

In [40]:
fig = go.Figure()
fig.add_scatter(x = egamma["x"], y = egamma[" y"], name="Data")
fig.add_scatter(x = egamma_xs, y = egamma_spl(egamma_xs), name="Interpolation")
fig.add_scatter(x = egamma_xs, y = egamma_ideal(egamma_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = egamma_xs, y = 1.0 - exp_cdf(egamma_xs, egamma_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "e-gamma L1T efficiency",
                  xaxis_title = "Momentum (GeV)",
                  yaxis_title = "Efficiency",
                  )


## Fit to Tau 

In [15]:
tau_xs = data_range(tau["x"])
tau_spl = PchipInterpolator(tau["x"], tau[" y"])

In [52]:
#fit to tau - minimize the difference between the trigger efficiency multiplied by the underlying distribution and the trigger rate
tau_fit = lambda l: np.abs(tau_rate - quad(lambda x: exp_dist(x, l) * tau_spl(x), 0, 100)[0])

In [53]:
tau_ideal = lambda x: 1.0 * (x > 38)

In [54]:
tau_soln = minimize_scalar(tau_fit)
tau_l = tau_soln.x

In [55]:
tau_l

0.04457408261926413

In [20]:
fig = go.Figure()
fig.add_scatter(x = tau["x"], y = tau[" y"], name = "Data")
fig.add_scatter(x = tau_xs, y = tau_spl(tau_xs), name = "Interpolation")
fig.add_scatter(x = tau_xs, y = tau_ideal(tau_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = tau_xs, y = 1.0 - exp_cdf(tau_xs, tau_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "Tau L1T Efficiency",
                  xaxis_title = "Momentum (GeV)",
                  yaxis_title = "Efficiency",
                  )
fig.update_xaxes(type = "linear")

## Fit to Jets

In [21]:
jet_xs = data_range(jet_energy["x"])
jet_spl = PchipInterpolator(jet_energy["x"], jet_energy[" y"])

In [109]:
jet_fit = lambda l: np.abs(jet_rate - quad(lambda x: exp_dist(x, l) * jet_ideal(x), 0, np.max(jet_xs))[0])
jet_soln = minimize_scalar(jet_fit, bounds = [0.0010, 0.0020], method="bounded")
jet_l = jet_soln.x

In [110]:
jet_l

0.001995202744121748

In [111]:
jet_ideal = lambda x: 1.0 * (x > 320)

In [112]:
fig = go.Figure()
fig.add_scatter(x = jet_energy["x"], y = jet_energy[" y"], name = "Data")
fig.add_scatter(x = jet_xs, y = jet_spl(jet_xs), name = "Interpolation")
fig.add_scatter(x = jet_xs, y = jet_ideal(jet_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = jet_xs, y = 1.0 - exp_cdf(jet_xs, jet_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "Jet Energy L1T Efficiency",
                  xaxis_title = "Momentum (GeV)"
                  )
fig.update_xaxes(type = "linear")

## Fit to Muon

In [83]:
muon_xs = data_range(muon["x"])
muon_spl = PchipInterpolator(muon["x"], muon[" y"])

In [89]:
muon_rate

0.364

In [100]:
#fit to muon - minimize the difference between the trigger efficiency multiplied by the underlying distribution and the trigger rate
muon_fit = lambda l: np.abs(muon_rate - quad(lambda x: exp_dist(x, l) * muon_spl(x), 0, np.max(muon_xs))[0])
muon_soln = minimize_scalar(muon_fit, bounds = [0.00, 0.100], method="bounded")
muon_l = muon_soln.x

/var/folders/qc/b4ggx3516_qc8fy84rgtp36c0000gq/T/ipykernel_62913/1988843803.py:2: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



In [103]:
muon_l

0.05513620525526383

In [104]:
muon_ideal = lambda x: 1.0 * (x > 22)

In [105]:
fig = go.Figure()
fig.add_scatter(x = muon["x"], y = muon[" y"], name = "data")
fig.add_scatter(x = muon_xs, y = muon_spl(muon_xs), name = "Interpolation")
fig.add_scatter(x = muon_xs, y = muon_ideal(muon_xs), name = "Ideal", line_dash="dash")
fig.add_scatter(x = muon_xs, y = 1.0 - exp_cdf(muon_xs, muon_l), name = "Data Fit")
fig.update_layout(width = 800,
                  height = 600,
                  title = "Single Muon L1T Efficiency",
                  xaxis_title = "Momentum (GeV)"
                  )
fig.update_xaxes(type = "linear")